In [1]:
import os
import numpy as np 
import pandas as pd
import arcpy
from arcpy import env

In [2]:
result_path = r"F:\cache_data\pre_soiltype_table\dy\cart\predict_csv\prediction_ph_NeuralNetFastAI.csv"

In [3]:
# 读取预测结果
data = pd.read_csv(result_path,index_col=False)

d:\Program Files\ArcGIS\Clone_gisenv\Clone\lib\site-packages\IPython\core\interactiveshell.py:3526: DtypeWarning: Columns (0,1,342,343) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
stander_path = r"D:\ArcGISProjects\workspace\shbyq\features_data_dy.gdb\DEM"

In [5]:
data.columns

Index(['prediction_0', 'prediction_1', 'prediction_2', 'prediction_3',
       'prediction_4', 'prediction_5', 'prediction_6', 'prediction_7',
       'prediction_8', 'prediction_9',
       ...
       'prediction_334', 'prediction_335', 'prediction_336', 'prediction_337',
       'prediction_338', 'prediction_339', 'prediction_340', 'prediction_341',
       'prediction_342', 'prediction_343'],
      dtype='object', length=344)

In [6]:
# 合并除第一列外的所有列到第一列
frames = []
for col in data.columns:
    frames.append(data[col])

# 合并 DataFrame
merged_data = pd.concat(frames, axis=0, ignore_index=True)

In [7]:
merged_data.dropna(inplace=True)

In [8]:
merged_data.size

171871183

In [9]:
merged_data.value_counts()

1f     159060589
12f      2551851
13d      1279745
14d      1174992
101       917397
         ...    
32           401
22           284
34           242
91           142
74d           90
Length: 62, dtype: int64

In [20]:

def label_array(mixed_array):
    # 获取唯一的值
    unique_values = list(set(merged_data.values))

    # 创建一个字典来映射唯一值到标签
    label_mapping = {value: index for index, value in enumerate(unique_values)}

    # 使用字典映射函数将数组标签化
    labeled_array = np.vectorize(label_mapping.get)(mixed_array)
    return labeled_array,label_mapping

In [21]:
# 标签化array
cach_result = label_array(merged_data)

In [22]:
type(merged_data)

pandas.core.series.Series

In [23]:
dem_array = arcpy.RasterToNumPyArray(stander_path)
dem_array.shape

(12881, 13343)

In [24]:
dem_array.shape[0] * dem_array.shape[1]

171871183

In [47]:
raster_array = np.reshape(cach_result[0],dem_array.shape)

array([[52, 52, 52, ..., 50, 50, 50],
       [50, 50, 50, ..., 50, 50, 50],
       [50, 50, 50, ..., 50, 50, 50],
       ...,
       [50, 50, 50, ..., 50, 50, 50],
       [50, 50, 50, ..., 50, 50, 50],
       [50, 50, 50, ..., 50, 50, 50]])

In [48]:
dem_array.dtype,raster_array.dtype

dtype('float32')

In [52]:
env.workspace = r"D:\ArcGISProjects\workspace\shbyq\feature_soiltype_result.gdb"
env.extent = stander_path

In [53]:
# 掩膜提取
def mask_raster(array,mask_ele,cell_size):
    out_raster = arcpy.NumPyArrayToRaster(
    array,
    arcpy.Point(arcpy.env.extent.XMin, arcpy.env.extent.YMin),
    cell_size,
    cell_size,
)
    """按掩膜提取栅格,空间参考设定为:CGCS2000_3_Degree_GK_CM_108E"""
    output_coordinate_system = arcpy.Describe(mask_ele).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=output_coordinate_system,snapRaster=mask_ele, cellSize=mask_ele):
        result_raster = arcpy.sa.ExtractByMask(out_raster, mask_ele, "INSIDE")
        return result_raster

In [54]:
# 按掩膜处理
result_raster = mask_raster(raster_array,stander_path, 5)
result_raster.save("prediction_sioltype_nni")
print("完成")

完成
